# Problem Noting:
## Camera Depth
Closer Person and Farther Person from the camera lead to different skeletons coordinate making the action detection model prone to changes in 3D environment. Since joint locations are detected in pixel coordinates, a person who is far away will have joint coordinates that appear compressed, while those who are closer will appear expanded.  
-> Solution: Normalize Person Coordinate and calculate joint distances.
+ Joint locations are Normalized using equation (1) where $(x_i, y_i)$ and $(x'_i, y'_i)$ are the original joint coordinate and normalized joint coordinate in i-th position. Thus the normalized joints reprepresent n features

# 0. Install and Import Dependencies

In [25]:
import cv2 as cv
import mediapipe as mp
import pandas as pd
import time
import os
import matplotlib.pyplot as plt

# 1. Image Preprocessing 

In [26]:
WIDTH = 640
HEIGHT = 640

In [27]:
def resize_and_show(img):
    h, w = img.shape[:2]
    if h < w:
        img = cv.resize(img, WIDTH, HEIGHT)

## 2. Make Detections

In [28]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

### Holistic Detection

In [14]:
cap = cv.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        #? Detect and Render
        # Recolor image (OpenCV use BGR, convert to RGB for processing)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False # block image writing capability  
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        #? Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
        #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        #                          )
        
        # 2. Right hand
        # mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        #                          mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
        #                          mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        #                          )

        # 3. Left Hand
        # mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
        #                          mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
        #                          mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        #                          )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv.imshow('Raw Webcam Feed', image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

In [15]:
results.pose_landmarks.landmark[0].visibility

0.9935827851295471

In [ ]:

# New variable name for the holistic model
holistic_model = mp.solutions.holistic
drawing_utils = mp.solutions.drawing_utils

# Specify the path to your image folder
image_folder = 'action_image'  # Replace with the actual path to your folder

# Get a list of all files in the image folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))] # You can add more image extensions if needed

# Initiate holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for image_file in image_files:
        # Construct the full path to the image
        image_path = os.path.join(image_folder, image_file)

        # Read the image using OpenCV
        frame = cv.imread(image_path)

        # Check if the image was successfully read
        if frame is None:
            print(f"Could not read image: {image_path}")
            continue

        #? Detect and Render
        # Recolor image (OpenCV use BGR, convert to RGB for processing)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False # block image writing capability

        # Make Detections
        results_test = holistic.process(image)
        # print(results_test.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        #? Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

        # Pose Detections
        drawing_utils.draw_landmarks(image, results_test.pose_landmarks, holistic_model.POSE_CONNECTIONS,
                                         drawing_utils.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                         drawing_utils.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                         )

        # Display the processed image
        cv.imshow(f'Processed Image: {image_file}', image)

        # Wait for a key press for each image (optional)
        cv.waitKey(0)

# Destroy all windows after processing all images
cv.destroyAllWindows()

In [19]:
results_test.pose_landmarks.landmark[0].visibility

0.9987233281135559

# 2. Capture Landmarks & Export to CSV

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [ ]:
import csv

In [21]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [22]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [23]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [ ]:
class_name = "assault_violence"

In [ ]:
import cv2 as cv
import mediapipe as mp
import os

# New variable name for the holistic model
holistic_model = mp.solutions.holistic
drawing_utils = mp.solutions.drawing_utils

# Specify the path to your image folder
image_folder = 'action_image'  # Replace with the actual path to your folder

# Get a list of all files in the image folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))] # You can add more image extensions if needed

# Initiate holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for image_file in image_files:
        # Construct the full path to the image
        image_path = os.path.join(image_folder, image_file)

        # Read the image using OpenCV
        frame = cv.imread(image_path)

        # Check if the image was successfully read
        if frame is None:
            print(f"Could not read image: {image_path}")
            continue

        #? Detect and Render
        # Recolor image (OpenCV use BGR, convert to RGB for processing)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False # block image writing capability

        # Make Detections
        results_test = holistic.process(image)
        # print(results_test.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        #? Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

In [ ]:
# New variable name for the holistic model
holistic_model = mp.solutions.holistic
drawing_utils = mp.solutions.drawing_utils

# Specify the path to your image folder
image_folder = 'image_frames'  # Replace with the actual path to your folder

# Get a list of all files in the image folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))] # You can add more image extensions if needed

# Initiate holistic model
with holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for image_file in image_files:
        # Construct the full path to the image
        image_path = os.path.join(image_folder, image_file)

        # Read the image using OpenCV
        frame = cv.imread(image_path)

        # Check if the image was successfully read
        if frame is None:
            print(f"Could not read image: {image_path}")
            continue

        #? Detect and Render
        # Recolor image (OpenCV use BGR, convert to RGB for processing)
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False # block image writing capability

        # Make Detections
        results_test = holistic.process(image)
        # print(results_test.face_landmarks)

        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks

        #? Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        
        
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concate rows
            row = pose_row
            
            # Append class name 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv.imshow('Raw Webcam Feed', image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()

## 3.1 Read in Collected Data and Process

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('coords.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df[df['class']=='assault_violence']

In [ ]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [ ]:
y_test

## 3.2 Train Machine Learning Classification Model

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [ ]:
fit_models

In [ ]:
fit_models['rc'].predict(X_test)

## 3.3 Evaluate and Serialize Model 